In [1]:
#Gui
#webScrapping


In [2]:
pip install requests

In [ ]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [3]:
import re
import requests
from bs4 import BeautifulSoup
from tkinter import Tk, Label, Button, Frame, StringVar
from tkinter.ttk import Combobox

In [4]:
class CricketScore:
    
    def __init__(self, rootWindow):
        self.rootWindow = rootWindow
        self.rootWindow.title('Live Cricket Score')
        self.rootWindow.geometry('800x500')

        # Set a simple background color instead of an image
        self.rootWindow.configure(bg='#FFFFFF')

        # Label for live matches
        self.label = Label(
            self.rootWindow, 
            text='Live Matches', 
            font=("times new roman", 60), 
            bg='#FFFFFF',  # Set background color to match window
            compound='center'
        )
        self.label.pack(padx=100, pady=50)
        
        # Fetch the live matches
        self.var = StringVar()
        self.matches = self.match_details()
        self.data = [i for i in self.matches.keys()]
        
        # Combobox to select a match
        self.cb = Combobox(self.rootWindow, values=self.data, width=50)
        self.cb.place(x=250, y=200)

        # Button to check score
        self.b = Button(
            self.rootWindow, 
            text="Check Score", 
            font=("times new roman", 14), 
            command=self.show_match_details
        )
        self.b.place(x=50, y=380)

    def show_match_details(self):
        # Create a frame to show match details
        self.frame1 = Frame(self.rootWindow, bg="#ADD8E6")
        self.frame1.place(x=180, y=280, width=600, height=200)

        match_name = self.cb.get()
        if match_name and match_name in self.matches:
            match_info = self.matches[match_name]

            Label(
                self.frame1, 
                text=match_name + " - " + match_info['match_header'], 
                font=("times new roman", 15, "bold"), 
                bg="#ADD8E6"
            ).place(x=20, y=15)
            
            Label(
                self.frame1, 
                text="Summary: " + match_info['summary'], 
                font=("times new roman", 12), 
                bg="#ADD8E6"
            ).place(x=20, y=50)
            
            Label(
                self.frame1, 
                text="Score: " + match_info['score_card'], 
                font=("times new roman", 12), 
                bg="#ADD8E6"
            ).place(x=20, y=80)
        else:
            Label(
                self.frame1, 
                text="No match selected or match details unavailable.", 
                font=("times new roman", 12), 
                bg="#ADD8E6"
            ).place(x=20, y=50)

    # Method to fetch match details
    def match_details(self):
        # Scrap the data
        details = self.scrap()
        # Parse and read elements / data
        live_match = {}
        for detail in details:
            live_team_details = {}
            summary = self.match_summary(detail)
            if summary is not None:
                match_header = self.match_header(detail).text
                teams = self.teams_name(detail)
                score_card = self.team_score(detail)
                live_team_details['summary'] = summary.text
                live_team_details['match_header'] = match_header
                live_team_details['score_card'] = score_card[0] + " :: " + score_card[1]
                live_match[teams[0] + " vs " + teams[1]] = live_team_details

        return live_match

    # Method to fetch the team score
    def team_score(self, detail):
        t = []
        team1_details = detail.find("div", class_="cb-hmscg-bat-txt").text
        team2_details = detail.find("div", class_="cb-hmscg-bwl-txt").text
        t.append(team1_details)
        t.append(team2_details)
        return t

    # Method to fetch the team names
    def teams_name(self, detail):
        t = []
        team1_details = detail.find("div", class_="cb-hmscg-bat-txt").text
        team1_index = re.search(r"\d", team1_details).start() if re.search(r"\d", team1_details) else len(team1_details)
        team2_details = detail.find("div", class_="cb-hmscg-bwl-txt").text
        team2_index = re.search(r"\d", team2_details).start() if re.search(r"\d", team2_details) else len(team2_details)
        t.append(team1_details[:team1_index].strip())
        t.append(team2_details[:team2_index].strip())
        return t

    # Method to fetch a given match summary
    def match_summary(self, detail):
        return detail.find("div", class_="cb-mtch-crd-state")

    # Method to fetch the match header
    def match_header(self, detail):
        return detail.find("div", class_="cb-mtch-crd-hdr")

    # Method to scrape the website Cricbuzz
    def scrap(self):
        URL = "https://www.cricbuzz.com/"
        page = requests.get(URL)

        # Parse the response and fetch the relevant data
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="match_menu_container")
        scrap_results = results.find_all("li", class_="cb-match-card")
        return scrap_results

# Main function to start the application
def main():
    rootWindow = Tk()
    obj = CricketScore(rootWindow)
    rootWindow.mainloop()

if __name__ == '__main__':
    main()